In [19]:
# import libs
import tensorflow as tf
import numpy as np

In [25]:
# The file path to save the data
save_file = 'model.ckpt'
saver = tf.train.Saver()

ValueError: No variables to save

In [26]:
# clear any prev
tf.reset_default_graph()

In [27]:
# get some data to play with
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [28]:
# vanilla ffnn
learning_rate = 0.001
training_epochs = 50
batch_size = 128
keep_prob = 0.5
display_step = 1
n_hidden1_layer = 256
n_hidden2_layer = 128

In [29]:
# inputs and labels
n_input = mnist.train.images[0].size
n_classes = mnist.train.labels[0].size

In [42]:
# dict of weights and biases
weights = {
    'hidden1_layer': tf.Variable(tf.random_normal([n_input, n_hidden1_layer])),
    'hidden2_layer': tf.Variable(tf.random_normal([n_hidden1_layer, n_hidden2_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden2_layer, n_classes]))
}
biases = {
    'hidden1_layer': tf.Variable(tf.random_normal([n_hidden1_layer])),
    'hidden2_layer': tf.Variable(tf.random_normal([n_hidden2_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [31]:
# features and labels
x = tf.placeholder("float", [None, 28, 28, 1])
x_flat = tf.reshape(x, [-1, n_input])
y = tf.placeholder("float", [None, n_classes])

In [32]:
# hidden layer 1
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden1_layer']), biases['hidden1_layer'])
layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.dropout(layer_1, keep_prob)

In [33]:
# hideen layer 2
layer_2 = tf.add(tf.matmul(layer_1, weights['hidden2_layer']), biases['hidden2_layer'])
layer_2 = tf.nn.relu(layer_2)
layer_2 = tf.nn.dropout(layer_2, keep_prob)

In [43]:
# output
logits = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])

In [41]:
# optimization objectives
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [36]:
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [37]:
# initialize net
init = tf.global_variables_initializer()

In [39]:
# build graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if epoch % 5 == 0:
            valid_accuracy = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(epoch, valid_accuracy))
    print("Optimization Finished!")
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.2734375
Epoch 5   - Validation Accuracy: 0.25
Epoch 10  - Validation Accuracy: 0.25
Epoch 15  - Validation Accuracy: 0.25
Optimization Finished!


NameError: name 'saver' is not defined

In [ ]:
# Launch the graph
with tf.Session() as sess:
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: mnist.test.images, labels: mnist.test.labels})
print('Test Accuracy: {}'.format(test_accuracy))